In [21]:
from pyscf import scf,gto
import numpy as np
import inspect
from FcMole import FcM
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

First compare gradient analytical from fcm with the one via finite differences on geometry


In [23]:
mol1=FcM(fcs=[.001,-.001],atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf1.init_guess_by_atom())
g1=mf1.Gradients()
g1.run()

converged SCF energy = -111.05722603438
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     1.0869705410
1 O     0.0000000000    -0.0000000000    -1.0869705410
----------------------------------------------


In [24]:
mol0=gto.M(atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")
mf0=scf.RHF(mol0)
mf0.scf()
g0=mf0.Gradients()
g0.kernel()

converged SCF energy = -111.064463936466
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     1.0871732099
1 O    -0.0000000000     0.0000000000    -1.0871732099
----------------------------------------------


array([[ 5.40838554e-17, -1.24257813e-15,  1.08717321e+00],
       [-5.40838554e-17,  1.24257813e-15, -1.08717321e+00]])

In [25]:
from alch_deriv import alch_deriv

The formula for the gradient is stated in Pople's article (Eq.21) as: 
$$ \frac{\partial E}{\partial x}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial x}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial x} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} c_{\mu i} \epsilon_i    c_{\nu i}^\dagger = S^{-1}FCOC^\dagger=S^{-1}FP
$$

In [26]:
(U,dP)=alch_deriv(mf0)
P=mf0.make_rdm1()
P1=mf1.make_rdm1()

First piece:
$$ \frac{\partial}{\partial Z} ( P \frac{\partial H^{(1)}}{\partial x})= 
\frac{\partial P}{\partial Z}\frac{\partial H^{(1)}}{\partial x}+ P \frac{\partial^2 H^{(1)}}{\partial x \partial Z}
$$
$\frac{\partial^2 H^{(1)}}{\partial x \partial Z}$ is trivially $\frac{\partial H^{(1)}}{\partial x}$ divided by the atom charge

In [27]:
ga=np.zeros((2,3))
ga[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)       #dP/dZ* dH/dx
ga[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
print(ga)
ga[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0)-g0.hcore_generator()(0),P)   #P* ddH/dZdx
ga[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1)-g0.hcore_generator()(1),P)
print(ga)

[[ 1.63185864e-19 -7.01318700e-18 -4.42827506e-03]
 [-1.63185864e-19  7.01318700e-18  4.42827506e-03]]
[[ 1.59585244e-19 -6.79440280e-18 -5.24933658e-03]
 [-1.59585244e-19  6.79440280e-18  5.24933658e-03]]


In [28]:
comp1=np.zeros((2,3,10,10))
comp1[0]=g1.hcore_generator()(0)-g0.hcore_generator()(0)
comp1[1]=g1.hcore_generator()(1)-g0.hcore_generator()(1)

In [29]:
comp2=np.zeros((2,3,10,10))
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += (g1.get_hcore()-g0.get_hcore())[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv

In [30]:
np.allclose(comp1,comp2)   #the difference lyes in  g.get_hcore()

True

g0.get_hcore() is the integral $<\chi_\mu |\nabla_r \hat{H}^{(1)}|\chi_\nu> $ is composed by two parts: the first refered to the kintic energy operator which is alchemy invariant, the second which has to be computed is refered to the nuclear electron attraction. <br>
To compute this we use moleintor.getints() using as arguments a mol environment (mol._env) with the added fractional charges and a mol._atm desription that show fractional charges.
Not forget to put a minus sign !!!! 

In [31]:
NUC_FRAC_CHARGE=gto.mole.NUC_FRAC_CHARGE
NUC_MOD_OF=gto.mole.NUC_MOD_OF
PTR_FRAC_CHARGE=gto.mole.PTR_FRAC_CHARGE
denv=mol0._env.copy()
datm=mol0._atm.copy()
fcs=[.001,-.001]
datm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
for i in range (mol0.natm):
    denv[datm[i,PTR_FRAC_CHARGE]]=fcs[i] 
dH1=-gto.moleintor.getints('int1e_ipnuc_sph',datm,mol0._bas,denv, None,3,0,'s1')   #minus sign !

In [32]:
comp2=np.zeros((2,3,10,10))
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += dH1[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv
np.allclose(comp1,comp2)

True

In [33]:
fdg=np.zeros((2,3))   #finite difference part 1 gradient
fdg[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0),mf1.make_rdm1())
fdg[0]-=np.einsum('xij,ij->x', g0.hcore_generator()(0),mf0.make_rdm1())
fdg[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1),mf1.make_rdm1())
fdg[1]-=np.einsum('xij,ij->x', g0.hcore_generator()(1),mf0.make_rdm1())
fdg

array([[-2.45041950e-15,  4.01313101e-14, -5.24514437e-03],
       [ 2.45041950e-15, -4.01313101e-14,  5.24514437e-03]])

In [34]:
ga_1=np.zeros((2,3))     # analytical gradient _part 1 
ga_1[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)
ga_1[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
#print(ga_1)
ga_1[0]+=np.einsum('xij,ij->x', comp2[0],P)
ga_1[1]+=np.einsum('xij,ij->x', comp2[1],P)
print(ga_1)

[[ 1.59585244e-19 -6.79440280e-18 -5.24933658e-03]
 [-1.59585244e-19  6.79440280e-18  5.24933658e-03]]


# Second piece:
$$\frac{\partial}{\partial Z} (P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma) )$$
here the two electron integral is invariant to alchemy, therefore is sufficient insert the density matrix derivative $dP$ in the following exression. 

In [35]:
#for the ref molecule
aoslices = mol0.aoslice_by_atom()
g2e_part2_0=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    g2e_part2_0[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P[p0:p1]) * 2)        #   P (Pd/dx(ml||ns))
g2e_part2_0    

array([[-4.90697390e-15,  3.48148877e-14,  1.17925050e+01],
       [ 4.90697390e-15, -3.48148877e-14, -1.17925050e+01]])

In [36]:
aoslices = mol1.aoslice_by_atom()
g2e_part2_1=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g1.get_veff(mol1, P1)
    g2e_part2_1[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P1[p0:p1]) * 2) 
g2e_part2_1

array([[-2.53144863e-15, -1.31071912e-15,  1.17969086e+01],
       [ 2.53144863e-15,  1.31071912e-15, -1.17969086e+01]])

In [94]:
#check the invariance:
np.allclose(g0.get_veff(mol0, P),g1.get_veff(mol1, P1))

False

In [96]:
print(np.allclose(g0.get_veff(mol0, P1),g1.get_veff(mol1, P1))) #update P
print(np.allclose(g0.get_veff(mol0, P+dP),g1.get_veff(mol1, P1))) #update P

True
True


In [99]:
aoslices = mol0.aoslice_by_atom()
ga_2=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    vhf_1 = g0.get_veff(mol0, P+dP)
    ga_2[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], dP[p0:p1]) * 2)
    ga_2[ia]+=(np.einsum('xij,ij->x',vhf_1[:,p0:p1]-vhf[:,p0:p1], P[p0:p1]) * 2)
ga_2

array([[-2.32817208e-19,  6.33185116e-18,  4.40672861e-03],
       [ 2.32817208e-19, -6.33185116e-18, -4.40672861e-03]])

In [100]:
np.allclose(ga_2,g2e_part2_1-g2e_part2_0,atol=1e-3)

True

In [101]:
g2e_part2_1-g2e_part2_0

array([[ 2.37552527e-15, -3.61256069e-14,  4.40354970e-03],
       [-2.37552527e-15,  3.61256069e-14, -4.40354970e-03]])

# Third piece:
$$-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
Luckily $S$ is invariant in alchemy, therefore the different in gradient is just:$$
-\sum_{\mu\nu}\frac{\partial W_{\mu\nu}}{\partial Z}\frac{\partial S_{\mu\nu}}{\partial x}
$$
### Obtaining derivatives of W
$$W=  \sum_i ^{mo.occ.} \epsilon_i C_{\mu i} C_{\nu i}^\dagger 
$$

$$ \frac{\partial W}{\partial Z_I}= \sum_i ^{mo.occ.} \left( \epsilon_i (CU)_{\mu i} C_{\nu i}^\dagger + 
\epsilon_i C_{\mu i} (CU)^\dagger_{\nu i}   +\frac{\partial \epsilon_i}{\partial Z_I} C_{\mu i} C_{\nu i}^\dagger \right)$$


$$ S W= S C\epsilon C^\dagger=FCC^\dagger= FP $$
$$W=  S^{-1}FP $$

In [43]:
"""  THE CODE IN g.grad_elec()
dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W
s1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx
for k, ia in enumerate(atmlst):
    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx
"""
pass

In [44]:
#verify that s1 is invariant
s1=g0.get_ovlp(mol0)
np.allclose(g0.get_ovlp(mol0),g1.get_ovlp(mol1))

True

In [45]:
#at first by finite differences 
W1=g1.make_rdm1e()
W0=g0.make_rdm1e()
fd_dW=W1-W0

In [46]:
# W0 can be constructed via C@np.diag(o*e)@C.T
o=mf0.mo_occ
O=np.diag(o)
e=mf0.mo_energy
C=mf0.mo_coeff
S=mf0.get_ovlp()
F=mf0.get_fock()
np.allclose(C@np.diag(e*o)@C.T,W0)

True

In [47]:
#to derive this expression first get dC as 
dC=C@U
#to get d(e) we need to get the fock hamiltonian and than get the new eigenvalues 
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])

In [48]:
np.allclose(mf1.get_fock()-F,dV+dF2el,atol=1e-5)

True

In [49]:
#try to get e in another way , using Roothans equations FC=SCe
print(np.allclose(np.linalg.inv(S)@F@(C),C@np.diag(e))) # S^-1 F C= C e 
print(np.allclose(np.linalg.inv(S)@F@P,W0))  #S^-1 F (C O C.T) = S^-1 F P = C e O C.T =W !!!!

True
True


In [50]:
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0
np.max(fd_dW-dW_a3)

7.673112843997743e-07

In [53]:
W1_contr=np.zeros((2,3))
ga_3=np.zeros((2,3))
W0_contr=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = mol0.aoslice_by_atom() [ia,2:]
    W1_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W1[p0:p1]) * 2
    ga_3[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], dW_a3[p0:p1]) * 2
    W0_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W0[p0:p1]) * 2
(ga_3),W1_contr-W0_contr

(array([[ 8.44921922e-16, -4.91622879e-15,  2.13537654e-05],
        [-4.85694102e-15,  4.96139048e-15, -2.07673277e-05]]),
 array([[-9.39716841e-17, -2.53336926e-15,  2.19504693e-05],
        [ 9.39716841e-17,  2.53336926e-15, -2.19504693e-05]]))

In [51]:
#W=S^-1F P is the way, cause of the orbital rotation
S=mf0.get_ovlp()
F=mf0.get_fock()
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0

In [103]:
ga_1+ga_2+ga_3  

array([[ 8.44848690e-16, -4.91669134e-15, -8.21254202e-04],
       [-4.85686779e-15,  4.96185303e-15,  8.21840639e-04]])

In [104]:
g1.grad_elec()-g0.grad_elec()

array([[-1.68865915e-16,  1.47233400e-15, -8.19644203e-04],
       [ 1.68865915e-16, -1.47233400e-15,  8.19644203e-04]])

## At the end the nuclear nuclear part 

In [61]:
"""def grad_nuc(mol, atmlst=None):
    gs = numpy.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 = mol.atom_charge(j)      <----------------------- derive here
        r2 = mol.atom_coord(j)
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i)     <----------------------- and here 
                r1 = mol.atom_coord(i)      
                r = numpy.sqrt(numpy.dot(r1-r2,r1-r2))
                gs[j] -= q1 * q2 * (r2-r1) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs
    """
pass

In [62]:
#is now easy to derive this function with respect to the nuclear charges
def alc_deriv_grad_nuc(mol,dL, atmlst=None):
    gs = np.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 =  mol.atom_charge(j) + dL[j]
        r2 = mol.atom_coord(j) 
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i) +dL[i]
                r1 = mol.atom_coord(i)
                r = np.sqrt(np.dot(r1-r2,r1-r2))
                gs[j] -= q1 * q2 * (r2-r1) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs

In [106]:
ga_4=alc_deriv_grad_nuc(mol0,[.001,-.001])-g0.grad_nuc()
ga_4

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [107]:
g1.grad_nuc()-g0.grad_nuc()

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [108]:
g0.grad_nuc()

array([[  0.        ,   0.        ,  14.81481481],
       [  0.        ,   0.        , -14.81481481]])

## Final comparison 

In [109]:
ga_1+ga_2+ga_3+ga_4

array([[ 8.44848690e-16, -4.91669134e-15, -2.04278893e-04],
       [-4.85686779e-15,  4.96185303e-15,  2.04865331e-04]])

In [110]:
gfd=g1.grad()-g0.grad()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     1.0869705410
1 O     0.0000000000    -0.0000000000    -1.0869705410
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     1.0871732099
1 O    -0.0000000000     0.0000000000    -1.0871732099
----------------------------------------------


array([[-1.68865915e-16,  1.47233400e-15, -2.02668894e-04],
       [ 1.68865915e-16, -1.47233400e-15,  2.02668894e-04]])

In [120]:
mol_nn=FcM(fcs=[1,-1],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_nn=scf.RHF(mol_nn)
mf_nn.scf(dm0=mf_nn.init_guess_by_atom())
g_nn=mf_nn.Gradients()
g_nn.run()

converged SCF energy = -105.439066087166
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     0.1755432102
1 O     0.0000000000    -0.0000000000    -0.1755432102
----------------------------------------------


In [121]:
mol_bf=FcM(fcs=[-1,1],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_bf=scf.RHF(mol_bf)
mf_bf.scf(dm0=mf_bf.init_guess_by_atom())
g_bf=mf_bf.Gradients()
g_bf.run()

converged SCF energy = -119.891952783639
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.5777071003
1 O    -0.0000000000     0.0000000000    -0.5777071003
----------------------------------------------


In [122]:
mol_bf=FcM(fcs=[0,0],atom="C 0 0 0; O 0 0 2.05",unit="Bohr",basis="STO-3G")
mf_bf=scf.RHF(mol_bf)
mf_bf.scf(dm0=mf_bf.init_guess_by_atom())
g_bf=mf_bf.Gradients()
g_bf.run()

converged SCF energy = -111.213679626751
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.2188663353
1 O    -0.0000000000    -0.0000000000    -0.2188663353
----------------------------------------------
